In [4]:
#39. Read this House price data to Pandas DataFrame.  

#Read the data
import pandas as pd
url = "https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/house%20price.csv"
df = pd.read_csv(url)
#df.head()

In [5]:
# #Create SQLite database and save the DataFrame there. 
import sqlite3

#Create a connection to SQLite database (create a database)
conn = sqlite3.connect('Housing.dp')

#Save dataframe to SQL database (create a table) [csv-file, Pandas dataframe and SQL table follow the same logics]
df.to_sql('USA_housing', conn, if_exists='replace',index = False)


4600

In [ ]:
#Let's test that it worked by querying the entire data
query = ''' SELECT
            *
            FROM USA_housing
            ''' 


In [ ]:
# Write a query which calculates the average price for each statezip. 